# Selenium para descargar base de dato de la intranet de Opción de Energía

## Librerías

In [21]:
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.keys import Keys
import pandas as pd
import glob
import os

## Page 1 - Login Opcion Comercializadora Intranet

In [2]:
driver = webdriver.Chrome("../Programas/chromedriver")

In [3]:
url = "https://canalagentes.opcionenergia.com/authentication/login"

In [178]:
driver.get(url)

In [179]:
driver.find_element_by_name("user").send_keys("vivolt")
driver.find_element_by_name("password").send_keys("temporal01vivolt")
driver.find_element_by_xpath("/html/body/app-root/app-agents/app-layout/app-login/section/div/div/div/div/form/div[2]/div[5]/div/div/div/button").send_keys(Keys.RETURN)

## Page 2 - Download "SIPS"

Para asignar la variable busco por el xpath, y desde tools de chrome, abro el codigo html y copio el xpath. 

Pulsando el boton de SIPS:

In [180]:
sips = driver.find_element_by_xpath('//*[@id="main_navbar"]/div[2]/div[1]/div[2]/ul[6]/li')

In [181]:
sips.click()

rellenando el CUPS: # agregar funcion para rellenar el cups desde la API

In [182]:
driver.find_element_by_name("Cups").send_keys("ES0021000005204736CV")

In [183]:
mostrar_grafico_sips = driver.find_element_by_xpath('//*[@id="pcoded"]/div[2]/div/div/div/div/div/div/div/app-sips/div[2]/div/form/div[2]/div/div[1]/button')

In [184]:
mostrar_grafico_sips.click()

In [185]:
descargar_csv = driver.find_element_by_xpath('//*[@id="pcoded"]/div[2]/div/div/div/div/div/div/div/app-sips/div[2]/div/form/div[2]/div/div[3]/button')

In [186]:
descargar_csv.click()

## Funciones para el proceso

In [22]:
import sys
%load_ext autoreload
%autoreload 2
sys.path.append('../configuration')
from config import login_data, client_cups, url

In [23]:
#send keys con valores escondidos en config. Por ahora datos insertados de forma manual. 

In [24]:
def download_consumo(login_data, client_cups, url):
    driver = webdriver.Chrome("../Programas/chromedriver")
    driver.get(url)
    sleep(1.5)
    driver.find_element_by_name("user").send_keys("vivolt")
    driver.find_element_by_name("password").send_keys("temporal01vivolt")
    sleep(0.5)
    driver.find_element_by_xpath("/html/body/app-root/app-agents/app-layout/app-login/section/div/div/div/div/form/div[2]/div[5]/div/div/div/button").send_keys(Keys.RETURN)
    sleep(1.5)
    driver.find_element_by_xpath('//*[@id="main_navbar"]/div[2]/div[1]/div[2]/ul[6]/li').click()
    sleep(0.5)
    driver.find_element_by_name("Cups").send_keys("ES0021000005204736CV")
    sleep(0.5)
    driver.find_element_by_xpath('//*[@id="pcoded"]/div[2]/div/div/div/div/div/div/div/app-sips/div[2]/div/form/div[2]/div/div[1]/button').click()
    sleep(0.5)
    driver.find_element_by_xpath('//*[@id="pcoded"]/div[2]/div/div/div/div/div/div/div/app-sips/div[2]/div/form/div[2]/div/div[3]/button').click()

In [25]:
download_consumo(login_data, client_cups, url)

## Import CSV

Sin excel:

In [44]:
def import_ConsumoCliente():
    list_of_files = glob.glob('/Users/pptrv/Downloads/*.csv')
    latest_file = max(list_of_files, key=os.path.getctime)
    data = pd.read_csv(latest_file, sep=";;", error_bad_lines = False)
    return(data)

<ipython-input-44-b9b1be04c718>:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv(latest_file, sep=";;", error_bad_lines = False)


,CUPS,ES0021000005204736CV
125,P1,2550
126,P2,7825
127,P3,738
128,P4,156
129,P5,457
130,P6,274


## Saving DF to CSV in Final_Proyect_Vivolt/Data/CSV

In [61]:
def saving_ConsumoCliente():
    df_sips = import_ConsumoCliente()
    df_sips_potencia = df_sips.head(7)
    df_sips_energia = df_sips.tail(6)
    df_sips_energia.to_csv("../Data/csv/bd_consumo_potencia_cliente.csv")
    df_sips_potencia.to_csv("../Data/csv/bd_consumo_potencia_cliente.csv")

In [62]:
saving_ConsumoCliente()

<ipython-input-44-b9b1be04c718>:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv(latest_file, sep=";;", error_bad_lines = False)
